# Worksheet 08

Name:  Medha Dhir
UID: U62101055

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [5]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.556496031818065, 5.764286128136619, 4.656735384460079, 5.757270144014049, 6.782340698535936, 4.460559336380519, 4.583605493928479, 5.950709071077135, 5.881246962820661, 5.425403794551165]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [6]:
mean = 8 
stdev = 1
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.614162243363462, 7.674408405475877, 7.7387481296850265, 8.7297743608764, 9.953206269670696, 5.613683905854343, 9.797273845464947, 5.429452035685413, 7.768480339668468, 8.152551378247475]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [7]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.152551378247475, 7.768480339668468, 5.425403794551165, 5.881246962820661, 5.429452035685413, 5.950709071077135, 9.797273845464947, 4.583605493928479, 5.613683905854343, 4.460559336380519]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

For each individual probability's

mean
variance
And a probaility of x being i-th species 

Gaussian Mixture Model: when all 
 are normal distribution with parameters:
 
 
Start with random 
Compute 
 for all Xi by using 
Compute / Update 
 from 
Repeat 2 & 3 until convergence

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [8]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) /(len(s2) + len(s1)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.425403794551165, 5.881246962820661, 5.429452035685413, 5.950709071077135, 4.583605493928479, 5.613683905854343, 4.460559336380519]
[8.152551378247475, 7.768480339668468, 9.797273845464947]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 5.334951514328246,  mean_2 = 8.572768521126962
var_1 = 0.30021175979768383,  var_2 = 0.7742917384452279


/Users/medhadhir/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [9]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.152551378247475
probability of observing that point if it came from cluster 0 =  9.908580579651856e-20
probability of observing that point if it came from cluster 1 =  0.44468018325202746
point =  7.768480339668468
probability of observing that point if it came from cluster 0 =  7.164619290724567e-15
probability of observing that point if it came from cluster 1 =  0.30040524930397644
point =  5.425403794551165
probability of observing that point if it came from cluster 0 =  1.2699014990344903
probability of observing that point if it came from cluster 1 =  0.00013308057967412285
point =  5.881246962820661
probability of observing that point if it came from cluster 0 =  0.2537702236473325
probability of observing that point if it came from cluster 1 =  0.0012250241399666595
point =  5.429452035685413
probability of observing that point if it came from cluster 0 =  1.264637567149157
probability of observing that point if it came from cluster 1 =  0.00013593724617826385
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [10]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([(p * (x - mean[0])**2) for p, x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([(p * (x - mean[1])**2) for p, x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 5.334397844422619,  mean_2 = 8.566811021261293
var_1 = 0.30017092602515827,  var_2 = 0.7884094003223018


The P and the means are quite similar, but there is a more notable difference in the variances.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [11]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  8.152551378247475
probability of observing that point if it came from cluster 0 =  9.623789527650094e-20
probability of observing that point if it came from cluster 1 =  0.4407652366581097
point =  7.768480339668468
probability of observing that point if it came from cluster 0 =  6.996404062116714e-15
probability of observing that point if it came from cluster 1 =  0.30304806647609905
point =  5.425403794551165
probability of observing that point if it came from cluster 0 =  1.2693506794296214
probability of observing that point if it came from cluster 1 =  0.0001805920038535714
point =  5.881246962820661
probability of observing that point if it came from cluster 0 =  0.25283981455035553
probability of observing that point if it came from cluster 1 =  0.0015297320181676453
point =  5.429452035685413
probability of observing that point if it came from cluster 0 =  1.264056157095872
probability of observing that point if it came from cluster 1 =  0.0001843223829081663
point =  

we see that it comes from C_1.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [12]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [5.425403794551165, 5.881246962820661, 5.429452035685413, 5.950709071077135, 4.583605493928479, 5.613683905854343, 4.460559336380519]
points in cluster 1 : [8.152551378247475, 7.768480339668468, 9.797273845464947]
